In [1]:
%pip install langchain
%pip install langchain-community
%pip install langchain-huggingface
%pip install chromadb
%pip install sentence-transformers



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install pandas


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
#takes the raw text (the book descriptions) and converts it into a format LangChain can work with.
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv
import pandas as pd
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.retrievers import ContextualCompressionRetriever

In [4]:
load_dotenv()

True

In [5]:
books = pd.read_csv("books_cleansed.csv")

In [6]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critic...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' ...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroin..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lo...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. L..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To Nort...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a m...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Pass...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless...


In [7]:
books.columns

Index(['isbn13', 'isbn10', 'title', 'authors', 'categories', 'thumbnail',
       'description', 'published_year', 'average_rating', 'num_pages',
       'ratings_count', 'title_and_subtitle', 'tagged_description'],
      dtype='object')

In [8]:
books["tagged_description"]

0       9780002005883  A NOVEL THAT READERS and critic...
1       9780002261982  A new 'Christie for Christmas' ...
2       9780006178736  A memorable, mesmerizing heroin...
3       9780006280897  Lewis' work on the nature of lo...
4       9780006280934  "In The Problem of Pain, C.S. L...
                              ...                        
5192    9788172235222  On A Train Journey Home To Nort...
5193    9788173031014  This book tells the tale of a m...
5194    9788179921623  Wisdom to Create a Life of Pass...
5195    9788185300535  This collection of the timeless...
5196    9789027712059  Since the three volume edition ...
Name: tagged_description, Length: 5197, dtype: object

In [9]:
books["tagged_description"] = (
    books["isbn13"].astype(str) + " " + books["description"].fillna("").astype(str)
)

books["tagged_description"].to_csv("tagged_description.txt", sep="\n", index=False, header=False)


In [10]:
# load the text file
raw_documents = TextLoader("tagged_description.txt").load()
"""chunk_size to a large value (e.g., 10000) so that Langchain won’t arbitrarily cut descriptions.
The separator="\n" ensures it splits only on newlines (each description is one line in your file).
chunk_overlap=0 keeps chunks fully independent.""" 
text_splitter = CharacterTextSplitter(chunk_size = 10000, chunk_overlap = 0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

In [11]:
print(len(documents))
print(documents[0])

272
page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the 

In [12]:
# Creating the database
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db_books = Chroma.from_documents(documents, embedding_model)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# query = "A book to teach children about nature."
# docs = db_books.similarity_search(query,k=10)

In [14]:
# docs

In [15]:
# retrieval (get a wider pool for reranking)
# LangChain sugar for turning your vector database (Chroma) into a Retriever object that can be
# converts your raw vector DB into a standard Retriever object. 
# that LangChain can use in chains/agents. “fetch the top 50 candidates”, which you can then rerank or filter.
retriever = db_books.as_retriever(search_type="similarity", search_kwargs={"k": 50})
# reranker model that scores (query, doc) pairs to the retirved 50 documents.
cross_encoder = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")
# keep best 10 after rerank
compressor = CrossEncoderReranker(model=cross_encoder, top_n=10)  
# LangChain utility that wraps your base retriever with a compressor (reranker).
ccr = ContextualCompressionRetriever(
    base_retriever=retriever,
    base_compressor=compressor,
)


In [16]:
query = "a book to teach children about nature"
results = ccr.invoke(query)   # reranked, highest relevance first
results[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='"9780067575208 First published more than three decades ago, this reissue of Rachel Carson\'s award-winning classic brings her unique vision to a new generation of readers. Stunning new photographs by Nick Kelsh beautifully complement Carson\'s intimate account of adventures with her young nephew, Roger, as they enjoy walks along the rocky coast of Maine and through dense forests and open fields, observing wildlife, strange plants, moonlight and storm clouds, and listening to the ""living music"" of insects in the underbrush. ""If a child is to keep alive his inborn sense of wonder."" Writes Carson, ""he needs the companionship of at least one adult who can share it, rediscovering with him the joy, excitement and mystery of the world we live in."" The Sense of Wonder is a refreshing antidote to indifference and a guide to capturing the simple power of discovery that Carson views as essential to life. In her insightful

In [25]:
isbn_str = results[0].page_content.split()[0].strip().replace('"', "")
isbn_num = int(isbn_str)
isbn_num

9780067575208

In [26]:
books[books["isbn13"] == int(results[0].page_content.split()[0].strip().replace('"', ""))]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
442,9780067575208,006757520X,The Sense of Wonder,Rachel Carson,Nature,http://books.google.com/books/content?id=Zee5S...,"First published more than three decades ago, t...",1998.0,4.39,112.0,1160.0,The Sense of Wonder,9780067575208 First published more than three ...


In [37]:
def retrieve_semantic_reccomendations(query:str, top_k:int=10) -> pd.DataFrame:
    results = ccr.invoke(query)[:top_k]  
    books_list = []
    for i in range(0, len(results)):
        books_list += [int(results[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)]

In [38]:
output = retrieve_semantic_reccomendations("Books based on ghosts")

In [39]:
output

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
306,9780060932664,006093266X,Collected Novellas,Gabriel Garcia Marquez,Fiction,http://books.google.com/books/content?id=JRcVu...,"Renowned as a master of magical realism, Gabri...",1999.0,4.01,288.0,822.0,Collected Novellas,9780060932664 Renowned as a master of magical ...
562,9780140128468,0140128468,Love's Executioner,Irvin D. Yalom,Existential psychotherapy,http://books.google.com/books/content?id=ZBptQ...,Contains eight case-studies of people the auth...,1991.0,4.20,270.0,216.0,Love's Executioner: And Other Tales of Psychot...,9780140128468 Contains eight case-studies of p...
709,9780140620474,0140620478,Far from the Madding Crowd,Thomas Hardy,England,http://books.google.com/books/content?id=6K8FZ...,Independent and spirited Bathsheba Everdene ha...,1994.0,3.94,374.0,312.0,Far from the Madding Crowd,9780140620474 Independent and spirited Bathshe...
1489,9780345439499,034543949X,The Best of Lester Del Rey,Lester Del Rey,Fiction,http://books.google.com/books/content?id=3LZ6P...,A collection of classic stories by the cofound...,2000.0,3.80,320.0,73.0,The Best of Lester Del Rey,9780345439499 A collection of classic stories ...
2237,9780441014224,0441014224,The God in the Moon,Richard A. Knaak,Fiction,http://books.google.com/books/content?id=ussPy...,Nermesa Klandes joins the Aquilonian army in h...,2006.0,3.72,297.0,57.0,The God in the Moon,9780441014224 Nermesa Klandes joins the Aquilo...
2705,9780553377767,0553377760,The Fairy Tales of Hermann Hesse,Hermann Hesse,Fiction,http://books.google.com/books/content?id=3_hvD...,A collection of twenty-two fairy tales by the ...,1995.0,3.99,266.0,3182.0,The Fairy Tales of Hermann Hesse,9780553377767 A collection of twenty-two fairy...
2855,9780586071397,0586071393,Faerie Tale,Raymond E. Feist,Fairy tales,http://books.google.com/books/content?id=Msdac...,A contemporary fantasy novel about the Hasting...,1989.0,3.87,490.0,7700.0,Faerie Tale,9780586071397 A contemporary fantasy novel abo...
2974,9780671024185,0671024183,Morgan's Run,Colleen McCullough,Fiction,http://books.google.com/books/content?id=XtgXa...,Colleen McCullough captivated millions with he...,2002.0,3.94,848.0,3394.0,Morgan's Run,9780671024185 Colleen McCullough captivated mi...
4270,9780963188922,0963188925,Pyramids of Montauk,Preston B. Nichols;Peter Moon,Fiction,http://books.google.com/books/content?id=SE8eA...,During WWII there was an attempt to achieve in...,1995.0,3.91,256.0,70.0,Pyramids of Montauk: Explorations in Conscious...,9780963188922 During WWII there was an attempt...
4860,9781591583660,1591583667,Crash Course in Web Design for Libraries,Charles P. Rubenstein,Computers,http://books.google.com/books/content?id=0n6FQ...,A handbook offers guidance in developing profe...,2007.0,2.67,196.0,9.0,Crash Course in Web Design for Libraries,9781591583660 A handbook offers guidance in de...


In [41]:
retrieve_semantic_reccomendations("mystery stories",2)